In [17]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
from config_parameters import DATA_TABLE_FILE, SUMMARY_SIMULATIONS_FILE, STRATEGIES_TABLE_FILE, SIMULATION_DATA_DICT, LOGS_FOLDER



def evaluate_trading_metrics(trades: pd.DataFrame, pnl_col: str = 'P&L') -> dict:
    """
    Calculates basic metrics for a trading strategy:
    - Profit Factor
    - Total P&L
    - Number of trades

    Parameters:
    trades : pd.DataFrame
        DataFrame containing the trade history with a P&L column.
    pnl_col : str
        Name of the column representing profit/loss for each trade.

    Returns:
        dict containing the calculated metrics.
    """
    if pnl_col not in trades.columns:
        return {
            'Profit Factor': 0,
            'P&L': 0,
            'Number of Trades': 0
    }
    pnl = trades[pnl_col]

    # Profit Factor
    total_profit = pnl[pnl > 0].sum()
    total_loss = pnl[pnl < 0].sum()
    profit_factor = total_profit / abs(total_loss) if total_loss != 0 else np.inf

    # Total P&L
    total_pnl = pnl.sum()

    return {
        'Profit Factor': round(profit_factor, 4),
        'P&L': round(total_pnl, 2),
        'Number of Trades': len(trades)
    }

def calculate_max_drawdown(account_history: pd.DataFrame, value_col: str = 'account_value'):
    """
    Calculates the maximum drawdown as a percentage, and net profit/loss.

    Parameters:
    - account_history: DataFrame with a column representing account value over time.
    - value_col: name of the column representing account value.

    Returns:
    - Max Drawdown (%) as a negative value.
    - Net P&L (difference between final and initial account value)
    """
    values = account_history[value_col]
    peak = values.cummax()
    drawdown = (values - peak) / peak
    max_drawdown = drawdown.min()
    return round(max_drawdown * 100, 2), round(values.iloc[-1] - values[0], 2)

def generate_metrics(trades, history):
    """
    Wrapper function that generates all key trading metrics.

    Returns:
    - A dictionary of profit factor, total P&L, number of trades,
      max drawdown, net P&L after fees, and ROI (placeholder for annual ROI)
    """
    metrics = evaluate_trading_metrics(trades)
    metrics['Max Drawdown'], metrics['P&L after fees'] = calculate_max_drawdown(history)
    metrics['ROI(y)'] = None  # Placeholder, can be calculated if needed

    return metrics

def calculate_roi(pl, days, initial_deposit):
    """
    Calculate the annualized Return on Investment (ROI).

    Parameters:
    - pl: Profit or loss after fees.
    - days: Number of days in the investment period.
    - initial_deposit: Starting capital used in the strategy.

    Returns:
    - ROI as a percentage, annualized.
    """
    return round(pl / initial_deposit / (days / 365) * 100, 2)

# Ordered columns for final summary output
ordered_cols = ['Strategy', 'Description', 'Ticker', 'Interval',
                'Profit Factor', 'P&L', 'Number', 'P&L after fees', 'Max Drawdown',
                'Start_date', 'End_date', 'length(d)', 'ROI(% year)', 'Deposit', 'Category', 'Okres']

def generate_new_summary(strategy, description, ticker, interval, metrics,
                         start_date, end_date, initial_deposit, category, period_id):
    """
    Generate a summary row for a specific strategy, interval, and ticker.

    Parameters:
    - strategy: Strategy ID.
    - description: Description of the strategy.
    - ticker: Instrument ticker.
    - interval: Time interval used (e.g., '1d', '1h').
    - metrics: Dictionary of calculated metrics.
    - start_date: Simulation start date.
    - end_date: Simulation end date.
    - initial_deposit: Starting balance.
    - category: Market category (e.g., 'CRYPTO').
    - period_id: Unique identifier for the time period.

    Returns:
    - DataFrame with one row, ordered and ready for summary table.
    """
    metrics['Strategy'] = strategy
    metrics['Interval'] = interval
    metrics['Ticker'] = ticker
    metrics['Description'] = description
    metrics['Start_date'] = start_date
    metrics['End_date'] = end_date
    metrics['length(d)'] = (end_date - start_date).days
    metrics['ROI(% year)'] = calculate_roi(metrics['P&L after fees'], metrics['length(d)'], initial_deposit)
    metrics['Deposit'] = initial_deposit
    metrics['Category'] = category
    metrics['Okres'] = period_id

    return pd.DataFrame([metrics])[ordered_cols]



In [ ]:
# Constructing file paths and processing simulation data

# Load market data and strategies
market_data_df = pd.read_excel(DATA_TABLE_FILE)
strategies_df = pd.read_excel(STRATEGIES_TABLE_FILE)

simulation_data_dir = SIMULATION_DATA_DICT


def generate_summary_for_simulations(start_date, end_date):
    # Full path to the specific simulation folder
    simulation_data_specific = os.path.join(simulation_data_dir, f"{start_date}_to_{end_date}")

    # Create directory if it doesn't exist
    os.makedirs(SUMMARY_SIMULATIONS_FILE, exist_ok=True)

    # Output path for the summary sheet
    sheet_output = os.path.join(SUMMARY_SIMULATIONS_FILE, f"{start_date}_to_{end_date}.xlsx")

    # Define intervals to be processed
    intervals = ['1w', '3d', '2d', '1d', '12h', '6h', '4h', '2h', '1h', '30m', '15m']

    start_date = pd.to_datetime(start_date)

    # Load or initialize summary DataFrame
    if os.path.exists(sheet_output):
        summary_df = pd.read_excel(sheet_output)
        print(f"✅ Loaded existing file: {sheet_output}")
    else:
        summary_df = pd.DataFrame(columns=ordered_cols)
        print(f"📄 Starting new summary file: {sheet_output}")

    # Loop through intervals and strategies
    for interval in intervals:
        for idx, strategy_row in strategies_df.iterrows():
            strategy_id = str(strategy_row['Numer'])
            description = strategy_row['Wskaźniki']

            for index, ticker_row in market_data_df.iterrows():
                ticker = ticker_row['Ticker']
                ticker_file = ticker_row['File']
                category = ticker_row["Category"]

                already_exists = summary_df[
                    (summary_df['Strategy'] == float(strategy_id)) &
                    (summary_df['Description'] == description) &
                    (summary_df['Ticker'] == ticker) &
                    (summary_df['Interval'] == interval)
                ]

                if not already_exists.empty:
                    continue  # Already added, skip
                
                trades_path = f"{simulation_data_specific}/{ticker} - strategy {strategy_id}, interval {interval} - trades.xlsx"
                history_path = f"{simulation_data_specific}/{ticker} - strategy {strategy_id}, interval {interval} - history.xlsx"
                print(history_path)
                # Check if simulation result files exist
                if not os.path.exists(history_path) or not os.path.exists(trades_path):
                    continue  # Skip if files are missing

                start_date = pd.to_datetime(ticker_row['Początek'])
                end_date = pd.to_datetime(ticker_row['Koniec'])

                # Load simulation results
                history_df = pd.read_excel(history_path)
                trades_df = pd.read_excel(trades_path)
                print(trades_df)
                # Adjust date range based on data
                start_date = history_df['timestamp'].iloc[0]
                end_date = history_df['timestamp'].iloc[-1]
                initial_value = history_df['account_value'].iloc[0]

                # Compute performance metrics
                metrics = generate_metrics(trades_df, history_df)

                # Generate a new summary row
                new_row = generate_new_summary(strategy_id, description, ticker, interval, metrics, start_date, end_date, initial_value, category, period_id)

                print(f"✅ Added {ticker}, interval: {interval}")
                summary_df = pd.concat([summary_df, new_row], ignore_index=True)

    # Save updated summary to file
    summary_df.to_excel(sheet_output, index=False)



start_date = "2024-01-01"
end_date = "2025-02-01"
#period_id = '3'
generate_summary_for_simulations(start_date, end_date)

NameError: name 'output_folder' is not defined